In [1]:
import tensorflow as tf
import numpy as np
from hourglass_tiny import HourglassModel
import configparser
import os
import pandas as pd
from keras.preprocessing import image
import scipy.io
from scipy.misc import imresize
import matplotlib.pyplot as plt
from skimage import transform
import scipy.ndimage.interpolation
import random

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [2]:
# load image and mask
test_img = image.load_img('Sitting/img/image0.jpg',target_size=[256,256])
test_img_arr = image.img_to_array(test_img)
test_label = scipy.io.loadmat('Sitting/masks/image0.mat')
test_label = test_label['M']
test_label = imresize(test_label,[256,256],interp='nearest')
label_resize = test_label.astype('float32')
label_floor = np.floor(label_resize/18)

plt.figure()
plt.imshow(test_img)
plt.show()
plt.figure()
plt.imshow(label_floor)
plt.show()

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [3]:
temp = np.zeros([256,256,4])
temp[:,:,0:3] = test_img_arr
temp[:,:,3]= label_floor

In [7]:
try_im = image.random_rotation(temp,rg=45,row_axis=0,col_axis=1,channel_axis=2)

In [8]:
plt.figure()
plt.imshow(try_im[:,:,0:3]/256)
plt.show()
new_mask2 = try_im[:,:,3]
new_mask2_ar = np.around(new_mask2)
plt.figure()
plt.imshow(new_mask2)
plt.show()

In [ ]:
def augmentor_with_keras:

In [49]:
def augmentor(image_asarray,mask):
    # do data augmentation for image and mask at the same time(random rotations, scaling, 
    # and color jittering)
    temp = np.zeros([256,256,4])
    temp[:,:,0:3] = image_asarray
    temp[:,:,3]= mask
    rd_angele = random.randint(-20,20)
    new_rote_im = transform.rotate(temp,angle = rd_angele,resize=True)
    #new_rote_ma = transform.rotate(image_asarray,angle = rd_angele,resize=True)
    rd_scale = random.uniform(0.6,1.6)
    new_rescale_im = transform.rescale(new_rote_im,scale=rd_scale,mode='constant')
    #new_rescale_ma = transform.rescale(new_rote_ma,scale=rd_scale,mode='constant')
    if new_rescale_im.shape[0]>256:
    # random crop
        temp = temp = np.zeros([256,256,4])
        center_m = [new_rescale_im.shape[0]/2,new_rescale_im.shape[0]/2]
        x = center_m[0]+random.randint(-50,50)
        y = center_m[1]+random.randint(-50,50)
        temp = new_rescale_im[int(x-128):int(x+128),int(y-128):int(y+128),:]
        aug_res = temp
    else:
        temp = np.zeros([256,256,4])
        length = new_rescale_im.shape[0]
        jrange = 256-length
        rd_startx = random.randint(0,jrange)
        rd_starty = random.randint(0,jrange)
        temp[rd_startx:rd_startx+length,rd_starty:rd_starty+length,:] = new_rescale_im
        aug_res = temp
    return aug_res

In [50]:
aug_res = augmentor(test_img_arr,label_floor)
plt.figure()
plt.imshow(aug_res[:,:,0:3]/256)
plt.show()
new_mask2 = aug_res[:,:,3]
new_mask2_ar = np.around(new_mask2)
plt.figure()
plt.imshow(new_mask2)
plt.show()

In [54]:
part = np.zeros([256,256])
for i in range(0,256):
    for j in range(0,256):
        if new_mask2[i,j] == 2:
            part[i,j] = 1
plt.figure()
plt.imshow(part)
plt.show()

In [63]:
new_mask3 = transform.rescale(new_mask2,scale=0.25,mode='constant')

In [64]:
plt.figure()
plt.imshow(new_mask3)
plt.show()

In [69]:
part = np.zeros([64,64])
for i in range(0,64):
    for j in range(0,64):
        if new_mask3[i,j] == 12:
            part[i,j] = 1
plt.figure()
plt.imshow(part)
plt.show()

In [73]:
a = 0.1
print('acc:',a,'  ','iou:',a)

acc: 0.1    iou: 0.1
